### 카테고리별 지원 파싱 (보건의료지원)
보건의료지원 (190~212페이지) 파싱

In [5]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [6]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [7]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [8]:
def parse_health_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """보건의료지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "보건·의료 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "의료급여", "건강보험료 지원", "암환자 의료비 지원")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["질병유형", "본인부담률", "비고"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["1종", "5%", "의료급여"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 보건의료지원제도명만
- 대상: 질병유형, 소득기준, 연령 등 구체적 조건
- 내용: 지원금액, 지원범위, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """보건의료지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "보건·의료 지원" 제외
- 실제 보건의료지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "보건의료지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [9]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(190, 213))  # 보건의료지원 섹션 (190~212페이지)
health_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_health_support(page_text, page_num)
    
    if program:
        health_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f"{program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 190 파싱 중... (1/23)
건강보험제도 추출 완료 (표 0개)
페이지 191 파싱 중... (2/23)
건강보험 산정특례 추출 완료 (표 1개)
페이지 192 파싱 중... (3/23)
건강보험 차상위 추출 완료 (표 0개)
페이지 193 파싱 중... (4/23)
고혈압･당뇨병 의료지원 추출 완료 (표 0개)
페이지 194 파싱 중... (5/23)
의료급여 산정특례(중증질환 산정특례) 추출 완료 (표 0개)
페이지 195 파싱 중... (6/23)
보건의료지원 추출 완료 (표 1개)
페이지 196 파싱 중... (7/23)
의료급여 추출 완료 (표 1개)
페이지 197 파싱 중... (8/23)
의료급여 추출 완료 (표 1개)
페이지 198 파싱 중... (9/23)
의료급여 본인부담 면제 추출 완료 (표 1개)
페이지 199 파싱 중... (10/23)
선택의료급여기관 제도 추출 완료 (표 0개)
페이지 200 파싱 중... (11/23)
국가건강검진제도 추출 완료 (표 0개)
페이지 201 파싱 중... (12/23)
건강검진 추출 완료 (표 0개)
페이지 202 파싱 중... (13/23)
통합건강증진사업 추출 완료 (표 0개)
페이지 203 파싱 중... (14/23)
성인 암환자 의료비 지원 추출 완료 (표 0개)
페이지 204 파싱 중... (15/23)
소아 암환자 의료비 지원 추출 완료 (표 0개)
페이지 205 파싱 중... (16/23)
희귀질환자 의료비 지원 추출 완료 (표 0개)
페이지 206 파싱 중... (17/23)
부양가족 생활보호비 지원 추출 완료 (표 1개)
페이지 207 파싱 중... (18/23)
정신건강복지센터 이용 추출 완료 (표 0개)
페이지 208 파싱 중... (19/23)
중독관리통합지원센터 추출 완료 (표 0개)
페이지 209 파싱 중... (20/23)
자살예방센터 운영 추출 완료 (표 0개)
페이지 210 파싱 중... (21/23)
정신질환자 치료비 지원사업 추출 완료 (표 2개)
페이지 

In [10]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/보건의료지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(health_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(health_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 23개 프로그램
저장 위치: ../../data/outputs/보건의료지원_2025.json


In [11]:
# 결과 미리보기
for program in health_programs:
    table_count = len(program.get('tables', []))
    print(f"\n{program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    print(f"대상: {program['target'][:80]}...")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


건강보험제도 (페이지 190, 표 0개)
대상: 국내에 거주하는 전 국민(의료급여수급자 제외), 직장가입자 : 근로자 및 사용자, 공무원 및 교직원, 지역가입자 : 직장가입자와 그 피부양자를 ...
내용: 건강보험제도는 진찰･검사, 약제･치료재료의 지급, 처치･수술 기타의 치료, 예방･재활, 입원, 간호, 이송에 대한 요양급여를 제공하며, 만성신부...

건강보험 산정특례 (페이지 191, 표 1개)
대상: 암, 뇌혈관질환, 심장질환, 희귀질환, 중증난치질환, 결핵, 잠복결핵, 중증화상, 중증외상, 중증치매 질환을 가진 건강보험가입자 및 피부양자...
내용: 건강보험 산정특례는 암, 뇌혈관질환, 심장질환, 희귀질환, 중증난치질환, 결핵, 잠복결핵, 중증화상, 중증외상, 중증치매 질환을 가진 건강보험가...
표: ['구분', '특례기간', '본인부담률']...

건강보험 차상위 (페이지 192, 표 0개)
대상: 희귀질환, 중증난치질환, 중증질환자, 6개월 이상 치료를 받고 있거나 치료가 필요한 만성질환자, 18세 미만 아동청소년 중 소득인정액이 기준 중...
내용: 희귀질환, 중증난치질환, 중증질환자는 요양급여비(입원･외래) 면제, 기본식대의 20%만 부담하며, 만성질환자와 18세 미만 아동은 입원비용의 1...

고혈압･당뇨병 의료지원 (페이지 193, 표 0개)
대상: 주민등록상 사업지역 거주 30세 이상 고혈압･당뇨병 환자...
내용: 고혈압･당뇨병 환자를 대상으로 사업지역 거주자에게 병원진료일 알림서비스, 교육･상담 서비스, 65세 이상의 경우 의료비 지원을 제공합니다. 의료...

의료급여 산정특례(중증질환 산정특례) (페이지 194, 표 0개)
대상: 중증질환자(암환자, 뇌혈관질환자, 심장질환자, 중증화상환자, 중증외상환자), 희귀질환 및 중증난치성질환자, 결핵질환자...
내용: 의료급여 산정특례 등록자(암, 중증화상, 희귀질환 및 중증난치성질환, 결핵)의 경우 의료급여 절차 예외로 2차 또는 3차 의료급여기관에 의료급여...

보건의료지원 (페이